### This will crash the kernel the first time. This is expected - if you rerun from the start, it will work.

In [ ]:
!apt-get install -y git-core ffmpeg espeak-ng
!pip install -q condacolab
import condacolab
condacolab.install()
condacolab.check()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'git' instead of 'git-core'
espeak-ng is already the newest version (1.50+dfsg-10).
git is already the newest version (1:2.34.1-1ubuntu1.10).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
✨🍰✨ Everything looks OK!
✨🍰✨ Everything looks OK!


In [ ]:
!echo -e "Grab a cup a coffee and a slice of pizza...\n\n"
!conda install -y -c conda-forge montreal-forced-aligner=2.2.17 openfst=1.8.2 kaldi=5.5.1068 && \
    pip install torch==2.1.0 && \
    pip install tensorboard==2.16.2 && \
    pip install phonemizer==3.2.1 && \
    pip install torchaudio==2.1.0 && \
    pip install datasets==2.16.0 && \
    pip install torchmetrics==0.11.1 && \
    pip install torchvision==0.16.0 && \
    pip install huggingface_hub==0.22.2

!pip install -U git+https://git@github.com/facebookresearch/audiocraft#egg=audiocraft
!git clone https://github.com/jasonppy/VoiceCraft.git

Grab a cup a coffee and a slice of pizza...


Channels:
 - conda-forge
Platform: linux-64
Solving environment: \ | / - \ | / - done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.4.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - kaldi=5.5.1068
    - montreal-forced-aligner=2.2.17
    - openfst=1.8.2


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    aom-3.7.1                  |       h59595ed_0         2.6 MB  conda-forge
    atk-1.0-2.38.0             |       h04ea711_2         348 KB  conda-forge
    audioread-3.0.1            |  py310hff52083_1          36 KB  conda-forge
    baumwelch-0.3.7            |       h00ab1b0_5         376 KB  conda-forge
    biopython-1.79             |  py310h576

In [ ]:
!mfa model download dictionary english_us_arpa && \
mfa model download acoustic english_us_arpa


 INFO     Saved model to /root/Documents/MFA/pretrained_models/dictionary/english_us_arpa.dict, you 
          can now use english_us_arpa in place of dictionary paths in mfa commands.                 
 INFO     Saved model to /root/Documents/MFA/pretrained_models/acoustic/english_us_arpa.zip, you can
          now use english_us_arpa in place of acoustic paths in mfa commands.                       


In [ ]:
# import libs
# if this throws an error, something went wrong installing dependencies or changing the kernel above!
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["USER"] = "YOUR_USERNAME" # TODO change this to your username

import torch
import torchaudio
import numpy as np
import random

from VoiceCraft.data.tokenizer import (
    AudioTokenizer,
    TextTokenizer,
)

In [ ]:
# simply installing audiocraft breaks due to no config, so move the default into site-packages
%cd /content/VoiceCraft
!git clone https://github.com/facebookresearch/audiocraft.git
!mv audiocraft/config /usr/local/lib/python3.10/site-packages/
!rm -rf audiocraft

/content/VoiceCraft
Cloning into 'audiocraft'...
remote: Enumerating objects: 1451, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 1451 (delta 13), reused 21 (delta 7), pack-reused 1419
Receiving objects: 100% (1451/1451), 1.96 MiB | 4.84 MiB/s, done.
Resolving deltas: 100% (792/792), done.


In [ ]:
# load model, encodec, and phn2num
# # load model, tokenizer, and other necessary files
device = "cuda" if torch.cuda.is_available() else "cpu"
from VoiceCraft.models import voicecraft
# reload voicecraft
import importlib
importlib.reload(voicecraft)
from VoiceCraft.models import voicecraft
voicecraft_name="gigaHalfLibri330M_TTSEnhanced_max16s.pth" # or giga330M.pth, giga830M.pth
ckpt_fn =f"./pretrained_models/{voicecraft_name}"
encodec_fn = "./pretrained_models/encodec_4cb2048_giga.th"
if not os.path.exists(ckpt_fn):
    os.system(f"wget https://huggingface.co/pyp1/VoiceCraft/resolve/main/{voicecraft_name}\?download\=true")
    os.system(f"mv {voicecraft_name}\?download\=true ./pretrained_models/{voicecraft_name}")
if not os.path.exists(encodec_fn):
    os.system(f"wget https://huggingface.co/pyp1/VoiceCraft/resolve/main/encodec_4cb2048_giga.th")
    os.system(f"mv encodec_4cb2048_giga.th ./pretrained_models/encodec_4cb2048_giga.th")

ckpt = torch.load(ckpt_fn, map_location="cpu")
model = voicecraft.VoiceCraft(ckpt["config"])
model.load_state_dict(ckpt["model"])
model.to(device)
model.eval()
phn2num = ckpt['phn2num']
text_tokenizer = TextTokenizer(backend="es2peak")
audio_tokenizer = AudioTokenizer(signature=encodec_fn, device=device) # will also put the neural codec model on gpu


Dora directory: /tmp/audiocraft_YOUR_USERNAME
/usr/local/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [ ]:
# Prepare your audio
# point to the original audio whose speech you want to clone
# write down the transcript for the file, or run whisper to get the transcript (and you can modify it if it's not accurate), save it as a .txt file
orig_audio = "/content/VoiceCraft/demo/k3.wav"
orig_transcript = "90년대와 1960년대 경제 발전 속도는 눈부실만큼,"
# move the audio and transcript to temp folder
temp_folder = "./demo/temp"
os.makedirs(temp_folder, exist_ok=True)
os.system(f"cp {orig_audio} {temp_folder}")
filename = os.path.splitext(orig_audio.split("/")[-1])[0]
with open(f"{temp_folder}/{filename}.txt", "w") as f:
    f.write(orig_transcript)
# run MFA to get the alignment
align_temp = f"{temp_folder}/mfa_alignments"
os.system(f"mfa align -j 1 --clean --output_format csv {temp_folder} english_us_arpa english_us_arpa {align_temp}")
# # if the above fails, it could be because the audio is too hard for the alignment model, increasing the beam size usually solves the issue
# os.system(f"mfa align -j 1 --clean --output_format csv {temp_folder} english_us_arpa english_us_arpa {align_temp} --beam 1000 --retry_beam 2000")


0

In [ ]:
# take a look the csv file in VoiceCraft/demo/temp/mfa_alignment, decide which part of the audio to use as prompt
cut_off_sec = 2.4 # NOTE: according to forced-alignment file demo/temp/mfa_alignments/84_121550_000074_000000.csv, the word "common" stop as 3.01 sec, this should be different for different audio
target_transcript = "The pace of economic development in the 1990s and 1960s was dazzling. Today is my birthday."
# NOTE: 3 sec of reference is generally enough for high quality voice cloning, but longer is generally better, try e.g. 3~6 sec.
audio_fn = f"{temp_folder}/{filename}.wav"
info = torchaudio.info(audio_fn)
audio_dur = info.num_frames / info.sample_rate

assert cut_off_sec < audio_dur, f"cut_off_sec {cut_off_sec} is larger than the audio duration {audio_dur}"
prompt_end_frame = int(cut_off_sec * info.sample_rate)

# run the model to get the output
# hyperparameters for inference
codec_audio_sr = 16000
codec_sr = 50
top_k = 0
top_p = 0.8
temperature = 1
silence_tokens=[1388,1898,131]
kvcache = 1 # NOTE if OOM, change this to 0, or try the 330M model

# NOTE adjust the below three arguments if the generation is not as good
stop_repetition = 3 # NOTE if the model generate long silence, reduce the stop_repetition to 3, 2 or even 1
sample_batch_size = 2 # for gigaHalfLibri330M_TTSEnhanced_max16s.pth, 1 or 2 should be fine since the model is trained to do TTS, for the other two models, might need a higher number. NOTE: if the if there are long silence or unnaturally strecthed words, increase sample_batch_size to 5 or higher. What this will do to the model is that the model will run sample_batch_size examples of the same audio, and pick the one that's the shortest. So if the speech rate of the generated is too fast change it to a smaller number.
seed = 1 # change seed if you are still unhappy with the result

def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
seed_everything(seed)

decode_config = {'top_k': top_k, 'top_p': top_p, 'temperature': temperature, 'stop_repetition': stop_repetition, 'kvcache': kvcache, "codec_audio_sr": codec_audio_sr, "codec_sr": codec_sr, "silence_tokens": silence_tokens, "sample_batch_size": sample_batch_size}
from VoiceCraft.inference_tts_scale import inference_one_sample
concated_audio, gen_audio = inference_one_sample(model, ckpt["config"], phn2num, text_tokenizer, audio_tokenizer, audio_fn, target_transcript, device, decode_config, prompt_end_frame)

# save segments for comparison
concated_audio, gen_audio = concated_audio[0].cpu(), gen_audio[0].cpu()
# logging.info(f"length of the resynthesize orig audio: {orig_audio.shape}")


# display the audio
from IPython.display import Audio
print("concatenate prompt and generated:")
display(Audio(concated_audio, rate=codec_audio_sr))

print("generated:")
display(Audio(gen_audio, rate=codec_audio_sr))

# # save the audio
# # output_dir
# output_dir = "/home/pyp/VoiceCraft/demo/generated_tts"
# os.makedirs(output_dir, exist_ok=True)
# seg_save_fn_gen = f"{output_dir}/{os.path.basename(audio_fn)[:-4]}_gen_seed{seed}.wav"
# seg_save_fn_concat = f"{output_dir}/{os.path.basename(audio_fn)[:-4]}_concat_seed{seed}.wav"

# torchaudio.save(seg_save_fn_gen, gen_audio, codec_audio_sr)
# torchaudio.save(seg_save_fn_concat, concated_audio, codec_audio_sr)

# if you get error importing T5 in transformers
# try
# pip uninstall Pillow
# pip install Pillow
# you are might get warnings like WARNING:phonemizer:words count mismatch on 300.0% of the lines (3/1), this can be safely ignored

concatenate prompt and generated:


generated:
